# Benchmark

In [ ]:
using Revise
using OrdinaryDiffEq
using HyperbolicDiffEq; HCL = HyperbolicDiffEq

balance_law = HyperbolicDiffEq.Euler{Float64,2}(1.4)

ϱ₀(x,y)  = 1.
vx₀(x,y) = sin(x)*cos(y)
vy₀(x,y) = -cos(x)*sin(y)
p₀(x,y)  = 100/1.4 + (cos(2x)+cos(2y))*3/16

u₀(x,y) = conserved_variables(ϱ₀(x,y), vx₀(x,y), vy₀(x,y), p₀(x,y), balance_law)
xmin=0.; xmax=2π; ymin=0.; ymax=2π

#fvol = CentralFlux()
#fvol = MorinishiFlux()
#fvol = DucrosEtAlFlux()
fvol = KennedyGruberFlux()
#fvol = PirozzoliFlux()
#fvol = ChandrashekarFluxEC()
#fvol = IsmailRoeFluxEC()
#fvol = RanochaFluxECandKEP()

fnum = fvol
#fnum = SuliciuFlux()
#fnum = LocalLaxFriedrichsFlux()

p = 9
Nx = 5
Ny = 5

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

tspan = (0., 14.)
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

#@time sol = solve(ode, SSPRK33(), dt=0.1/((2p+1)*max(Nx,Ny)), save_everystep=false);
tstops = range(tspan[1], tspan[end], length=100)
@time sol = solve(ode, SSPRK33(), dt=0.1/((2p+1)*max(Nx,Ny)), saveat=tstops, tstops=tstops);

In [ ]:
using BenchmarkTools

u = copy(ode.u0)
du = similar(u)
Pp1 = p+1
jacx = 2 / semidisc.meshx.Δx
jacy = 2 / semidisc.meshy.Δx

u1 = first(u)
u2 = last(u)
direction = Val{:x}()
t = tspan[1]


usethreads = Val{false}()
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, false)
display(usethreads)
#display(@benchmark HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads))
#@code_warntype HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads)
display(@benchmark HCL.add_flux_differences!(du, u, semidisc))
display(@benchmark HCL.add_numerical_fluxes!(du, u, semidisc))
display(@benchmark semidisc(du, u, nothing, t))

usethreads = Val{true}()
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
display(usethreads)
#display(@benchmark HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads))
#@code_warntype HCL.add_flux_differences_inner_loop!(du,u,balance_law,fvol,Nx,Ny,Pp1,basis.D,jacx,jacy,usethreads)
display(@benchmark HCL.add_flux_differences!(du, u, semidisc))
display(@benchmark HCL.add_numerical_fluxes!(du, u, semidisc))
display(@benchmark semidisc(du, u, nothing, t))


In [ ]:
using LaTeXStrings, Plots; pyplot()

npoints = 8p+5

xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display

In [ ]:
using MappedArrays

ϱ = mappedarray(u->u.ϱ, sol[end])
vx = mappedarray(u->begin ϱ,vx,vy,p=HCL.primitive_variables(u,balance_law); vx end, sol[end])
vy = mappedarray(u->begin ϱ,vx,vy,p=HCL.primitive_variables(u,balance_law); vy end, sol[end])
p = mappedarray(u->begin ϱ,vx,vy,p=HCL.primitive_variables(u,balance_law); p end, sol[end])

clim = [extrema(ϱ); extrema(p); extrema(vx); extrema(vy)]

xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
@gif for idx in eachindex(sol)
    HCL.evaluate_coefficients!(xplot, yplot, uplot, sol[idx], meshx, meshy, basis)
    plt = heatmap((xplot, yplot, uplot, balance_law), title=@sprintf("Time t = %6.3f", sol.t[idx]), cbar=true)
    for idx in 1:length(plt)
        plot!(plt[idx], clim=clim[idx])
    end
end

# Euler

In [ ]:
import HyperbolicDiffEq; #reload("HyperbolicDiffEq")
using OrdinaryDiffEq

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients
semidiscretise = HCL.semidiscretise; UniformPeriodicFluxDiffDisc2D = HCL.UniformPeriodicFluxDiffDisc2D
EnergyConservativeFlux = HCL.EnergyConservativeFlux; GodunovFLux = HCL.godunov
EulerVar2D = HCL.EulerVar2D; conserved_variables = HCL.conserved_variables; SuliciuFlux=HCL.SuliciuFlux
LocalLaxFriedrichsFlux = HCL.LocalLaxFriedrichsFlux; ChandrashekarFluxEC=HCL.ChandrashekarFluxEC
IsmailRoeFluxEC=HCL.IsmailRoeFluxEC; CentralFlux=HCL.CentralFlux; RanochaFluxECandKEP=HCL.RanochaFluxECandKEP
MorinishiFlux=HCL.MorinishiFlux; DucrosEtAlFlux=HCL.DucrosEtAlFlux; KennedyGruberFlux=HCL.KennedyGruberFlux
PirozzoliFlux=HCL.PirozzoliFlux
integrate=HCL.integrate; kinetic_energy=HCL.kinetic_energy; entropy=HCL.entropy

const γ = 1.4
balance_law = HCL.Euler{Float64,2}(γ)

ϱ₀(x,y)  = 1.
vx₀(x,y) = sin(x)*cos(y)
vy₀(x,y) = -cos(x)*sin(y)
p₀(x,y)  = 100/γ + (cos(2x)+cos(2y))*3/16

u₀(x,y) = conserved_variables(ϱ₀(x,y), vx₀(x,y), vy₀(x,y), p₀(x,y), balance_law)
xmin=0.; xmax=2π; ymin=0.; ymax=2π

#fvol = CentralFlux()
#fvol = MorinishiFlux()
#fvol = DucrosEtAlFlux()
#fvol = KennedyGruberFlux()
fvol = PirozzoliFlux()
#fvol = EnergyConservativeFlux()
#fvol = ChandrashekarFluxEC()
#fvol = IsmailRoeFluxEC()
#fvol = RanochaFluxECandKEP()

fnum = fvol
#fnum = SuliciuFlux()
#fnum = LocalLaxFriedrichsFlux()

p = 1
Nx = 25
Ny = 25

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

tspan = (0., 20.)
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

#solve(ode, SSPRK104(), dt=tspan[end]-tspan[1], save_everystep=false)
@time sol = solve(ode, SSPRK104(), dt=1/((2p+1)*max(Nx,Ny)), save_everystep=true);


In [ ]:
compute_ekin(u) = integrate(u->kinetic_energy(u,balance_law), u, meshx, meshy, basis)
compute_entr(u) = integrate(u->entropy(u,balance_law), u, meshx, meshy, basis)

ekin = map(compute_ekin, sol.u)
ekin1 = ekin[1]
entr = map(compute_entr, sol.u)
entr1 = entr[1]

fig_ekin = plot(sol.t, ekin/ekin1, label="", xguide=L"t", yguide="Rel. Kinetic Energy")
fig_entr = plot(sol.t, entr/entr1, label="", xguide=L"t", yguide="Rel. Entropy")

@show entr[1] - entr[end]
plot(fig_ekin, fig_entr)

In [ ]:
@show integrate(sol[1], meshx, meshy, basis)
@show integrate(sol[end], meshx, meshy, basis)

@show integrate(u->kinetic_energy(u,balance_law), sol[1], meshx, meshy, basis)
@show integrate(u->kinetic_energy(u,balance_law), sol[end], meshx, meshy, basis)

@show integrate(u->entropy(u,balance_law), sol[1], meshx, meshy, basis)
@show integrate(u->entropy(u,balance_law), sol[end], meshx, meshy, basis)

In [ ]:
npoints = 10p+3


xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
heatmap((xplot, yplot, uplot, balance_law), cbar=true) |> display


# Test Burgers'

In [ ]:
import HyperbolicDiffEq; reload("HyperbolicDiffEq")
using OrdinaryDiffEq

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients
semidiscretise = HCL.semidiscretise; UniformPeriodicFluxDiffDisc2D = HCL.UniformPeriodicFluxDiffDisc2D
EnergyConservativeFlux = HCL.EnergyConservativeFlux; GodunovFLux = HCL.godunov


balance_law = HCL.Burgers{Float64,2}()
u₀(x,y) = sinpi(x); xmin=0.; xmax=2.; ymin=-1.; ymax=1.

fvol = EnergyConservativeFlux()
fnum = GodunovFLux

p = 1
Nx = 50
Ny = 50

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

tspan = (0., 4.)
semidisc = UniformPeriodicFluxDiffDisc2D(balance_law, meshx, meshy, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

solve(ode, SSPRK104(), dt=tspan[end]-tspan[1], save_everystep=false)
@time sol = solve(ode, SSPRK104(), dt=1./((2p+1)*max(Nx,Ny)), save_everystep=false);


In [ ]:
npoints = 10p+3

xplot, yplot, uplot = evaluate_coefficients(sol[1], meshx, meshy, basis, npoints)
heatmap(xplot, yplot, uplot) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
heatmap(xplot, yplot, uplot) |> display

xplot, yplot, uplot = evaluate_coefficients(sol[end], meshx, meshy, basis, npoints)
plot(xplot, uplot[npoints÷2+1,:]) |> display

In [ ]:
using BenchmarkTools

reload("HyperbolicDiffEq")

u = copy(ode.u0)
du = zeros(u)
jacx = 2 / semidisc.meshx.Δx
jacy = 2 / semidisc.meshy.Δx
direction = Val{:x}()

#usethreads = Val{true}()
#usethreads = Val{false}()

#@benchmark HCL.add_flux_differences!(du, u, semidisc)
@benchmark HCL.add_numerical_fluxes!(du, u, semidisc)

#@benchmark fvol(1., 6., balance_law, direction)
#@code_native fvol(1., 6., balance_law, direction)
#@benchmark fnum(1., 6., balance_law, direction)
#@code_native fnum(1., 6., balance_law, direction)

#@code_warntype HCL.add_flux_differences!(du, u, semidisc, Val{true}())
#@code_warntype HCL.add_flux_differences_inner_loop!(du, u, balance_law, fvol, Nx, Ny, p+1, basis.D, jacx, jacy, Val{false}())

# Test Plotting

In [ ]:
import HyperbolicDiffEq; reload("HyperbolicDiffEq")

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients

u₀(x,y) = sinpi(x)*sinpi(y); xmin=-1.; xmax=1.; ymin=-1.; ymax=1.
#u₀(x,y) = (3x + y ^ 2) * abs(sin(x) + cos(y)); xmin=0.; xmax=20.; ymin=-0.; ymax=10.


p = 9
Nx = 10
Ny = 10

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
basis = LobattoLegendre(p)

u0 = compute_coefficients(u₀, meshx, meshy, basis)
xplot, yplot, uplot = evaluate_coefficients(u0, meshx, meshy, basis)


#fig = plot(xguide=L"x", yguide=L"y")
display("contour")
contour(xplot, yplot, uplot) |> display
display("contour(..., fill=true)")
contour(xplot, yplot, uplot, fill=true) |> display
display("heatmap")
heatmap(xplot, yplot, uplot) |> display
display("surface")
surface(xplot, yplot, uplot) |> display
display("contour3d")
contour3d(xplot, yplot, uplot) |> display
display("wireframe")
wireframe(xplot, yplot, uplot) |> display
display("wireframe(..., contours=true)")
wireframe(xplot, yplot, uplot, contours=true) |> display
